In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

indices<-Indices()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### fetch the latest rates across tenors

In [2]:
maxDt <- (indices$IndiaGsecTimeSeries() %>%
    summarize(MAX_TS = max(TIME_STAMP)) %>%
    collect())$MAX_TS[1]

indices$IndiaGsecTimeSeries() %>%
    filter(TIME_STAMP == maxDt) %>%
    arrange(NAME) %>%
    print()

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 7]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: NAME
  NAME  TIME_STAMP   TRI   PRI COUPON   YTM DURATION
  <chr> <date>     <dbl> <dbl>  <dbl> <dbl>    <dbl>
1 0_5   2019-07-10 2917.  836.   7.84  6.38     2.83
2 10_15 2019-07-10 2947.  913.   7.83  6.78     8.10
3 15_20 2019-07-10 2954.  907.   7.69  6.83     9.56
4 20_30 2019-07-10 2986.  906.   8.07  6.86    11.8 
5 5_10  2019-07-10 2934.  862.   7.85  6.71     5.63


### get all total-return NSE indices for the latest time_stamp

In [3]:
maxDtNse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtNse && NAME %like% '% TR%') %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                          
   <chr>                                         
 1 NIFTY 100 TR                                  
 2 NIFTY 200 TR                                  
 3 NIFTY 50 FUTURES TR INDEX                     
 4 NIFTY 50 TR                                   
 5 NIFTY 500 TR                                  
 6 NIFTY ADITYA BIRLA GROUP TR                   
 7 NIFTY ALPHA 50 TR                             
 8 NIFTY ALPHA LOW-VOLATILITY 30 TR              
 9 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR      
10 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR
# … with more rows


## get start and end dates for all total-return NSE indices

In [4]:
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(NAME %like% '% TR%') %>%
     summarize(ST=min(TIME_STAMP), ET=max(TIME_STAMP)) %>%
     print(n = 10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   NAME                                           ST         ET        
   <chr>                                          <date>     <date>    
 1 NIFTY 100 TR                                   2003-01-01 2019-07-10
 2 NIFTY 200 TR                                   2004-01-01 2019-07-10
 3 NIFTY 50 FUTURES TR INDEX                      2005-04-01 2019-07-10
 4 NIFTY 50 TR                                    1999-06-30 2019-07-10
 5 NIFTY 500 TR                                   1998-11-26 2019-07-10
 6 NIFTY ADITYA BIRLA GROUP TR                    2005-04-01 2019-07-10
 7 NIFTY ALPHA 50 TR                              2003-12-31 2019-07-10
 8 NIFTY ALPHA LOW-VOLATILITY 30 TR               2005-04-01 2019-07-10
 9 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR       2005-04-01 2019-07-10
10 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR 2005-04-01 2019-07-10
# … with more rows


### get all BSE indices for the latest time_stamp

In [5]:
maxDtBse <- (indices$BseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$BseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME  
   <chr> 
 1 BSE100
 2 BSE200
 3 BSE500
 4 BSEMID
 5 BSESML
 6 CARBON
 7 CPSE  
 8 DOL100
 9 DOLL30
10 DOLLEX
# … with more rows


### fetch the latest NSE NIFTY 50 constituents 

In [6]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == "NIFTY 50") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$NseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   NAME     TIME_STAMP SYMBOL     INDUSTRY           CAP_WEIGHT
   <chr>    <date>     <chr>      <chr>                   <dbl>
 1 NIFTY 50 2019-07-10 ADANIPORTS SERVICES                0.681
 2 NIFTY 50 2019-07-10 ASIANPAINT CONSUMER GOODS          1.30 
 3 NIFTY 50 2019-07-10 AXISBANK   FINANCIAL SERVICES      3.31 
 4 NIFTY 50 2019-07-10 BAJAJ-AUTO AUTOMOBILE              0.807
 5 NIFTY 50 2019-07-10 BAJAJFINSV FINANCIAL SERVICES      1.05 
 6 NIFTY 50 2019-07-10 BAJFINANCE FINANCIAL SERVICES      1.78 
 7 NIFTY 50 2019-07-10 BHARTIARTL TELECOM                 1.31 
 8 NIFTY 50 2019-07-10 BPCL       ENERGY                  0.632
 9 NIFTY 50 2019-07-10 BRITANNIA  CONSUMER GOODS          0.702
10 NIFTY 50 2019-07-10 CIPLA      PHARMA                  0.587
# … with more rows


### fetch the latest BSE SENSEX constituents    

In [7]:
maxDt <- (indices$BseConstituents() %>%
          filter(NAME == "sp bse sensex") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$BseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "sp bse sensex") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   NAME          TIME_STAMP   CODE SYMBOL     SECURITY_NAME                   
   <chr>         <date>      <int> <chr>      <chr>                           
 1 sp bse sensex 2019-07-01 500010 HDFC       Housing Development Finance Corp
 2 sp bse sensex 2019-07-01 500034 BAJFINANCE Bajaj Finance Ltd               
 3 sp bse sensex 2019-07-01 500112 SBIN       State Bank of India             
 4 sp bse sensex 2019-07-01 500180 HDFCBANK   HDFC Bank Ltd                   
 5 sp bse sensex 2019-07-01 500182 HEROMOTOCO Hero MotoCorp Ltd               
 6 sp bse sensex 2019-07-01 500209 INFY       Infosys Ltd                     
 7 sp bse sensex 2019-07-01 500247 KOTAKBANK  Kotak Mahindra Bank Ltd         
 8 sp bse sensex 2019-07-01 500295 VEDL       Vedanta Ltd                     
 9 sp bse sensex 2019-07-01 500312 ONGC       Oil & Natural Gas Corp Ltd      
10 sp bse sensex 2019-07-01 500325

### fetch index date ranges published on yahoo finance

In [8]:
indices$YahooFinanceTimeSeries() %>%
    group_by(NAME) %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    arrange(NAME) %>%
    print(n=Inf)

# Source:     lazy query [?? x 3]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockVizUs2]
# Ordered by: NAME
  NAME  ST         ET        
  <chr> <date>     <date>    
1 ^GSPC 1950-01-03 2019-07-05
2 ^MID  1981-01-02 2019-07-05
3 ^N225 1965-01-05 2019-07-05
4 ^NDX  1985-10-01 2019-07-05
5 ^RAG  2002-09-30 2019-07-05
6 ^RAV  2002-09-30 2019-07-05
7 ^RUA  1987-09-10 2019-07-05
8 ^RUT  1987-09-10 2019-07-05
9 ^VIX  1990-01-02 2019-07-05


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)